In [1]:
import pandas as pd
import csv
import torch
import copy
import numpy as np
from torchtext import data
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#based on homework 2 framework

In [2]:
data_raw = pd.read_csv('standardized.csv')
decisions = pd.read_csv('decisions.csv')

In [73]:
TEXT = data.Field(
    sequential=True,
    fix_length=500,
    tokenize='spacy',
    pad_first=True,
    lower=True
)

LABEL = data.LabelField(dtype = torch.float,
                        use_vocab=False, 
                        sequential=False,
                        is_target=True)

full = data.TabularDataset('standardized.csv', 'csv', skip_header=True,
        fields=[
            ('docket', None),
            ('outcome', None),
            ('facts', TEXT),
            ('conclusion', None),
            ('target', LABEL)
        ])

TEXT.build_vocab(
    full,
    max_size=20000,
    min_freq=150,
    vectors=None
)

LABEL.build_vocab(full)

In [74]:
print(vars(full.examples[125]))

{'facts': ['  ', 'the', 'subject', 'property', 'is', '116', 'years', 'old', ',', 'and', 'consists', 'of', 'a', 'three', '-', 'story', 'dwelling', 'of', 'frame', 'construction', 'containing', '2,068', 'square', 'feet', 'of', 'living', 'area', '.', ' ', 'features', 'of', 'the', 'home', 'include', 'a', 'full', 'docket', 'no', ':', '09', '-', '23425.001-r-1', '   ', '2', 'of', '4', 'basement', '.', ' ', 'the', 'subject', 'property', 'has', 'a', '3,712', 'square', 'foot', 'site', ',', 'is', 'located', 'in', 'lake', 'view', 'township', ',', 'cook', 'county', 'and', 'is', 'classified', 'as', 'a', 'class', '2', 'property', 'under', 'the', 'cook', 'county', 'real', 'property', 'assessment', 'classification', 'ordinance', '.', ' ', 'the', 'appellant', 'contends', 'assessment', 'inequity', 'as', 'the', 'basis', 'of', 'the', 'appeal', '.', ' ', 'in', 'support', 'of', 'this', 'argument', 'the', 'appellant', 'submitted', 'information', 'on', 'three', 'suggested', 'equity', 'comparables', '.', '  ', 

In [75]:
SEED = 189053
train_data, test_data, valid_data = full.split(split_ratio=[0.7, 0.15, 0.15], random_state = random.seed(SEED))

In [76]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 31921
Number of validation examples: 6840
Number of testing examples: 6841


In [77]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

#13114 unique @ min_freq 15
#8700 unique @ min_freq 25
#6186 unique @ min_freq 35
#4533 unique @ min_freq 50
#3352 unique @ min_freq 75
#2799 unique @ min_freq 100
#2173 unique @ min_freq 100

Unique tokens in TEXT vocabulary: 2173
Unique tokens in LABEL vocabulary: 2


In [78]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = 'cpu',
    sort_key=lambda x: len(x.facts))

In [80]:
def binary_accuracy(preds, y):
    """
    Return accuracy per batch
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

class WordEmbAvg(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)  
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()                                 
        
    def forward(self, text):
        embeddings = self.embedding(text)
        embeddings_avg = embeddings.mean(0)
        output = self.linear1(embeddings_avg)
        final = self.linear2(self.relu(output))
        return final
    
class TrainingModule():

    def __init__(self, model):
        self.model = model
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.model.train()
    
    def train_epoch(self, iterator):
        epoch_loss = 0
        epoch_acc = 0
    
        for batch in iterator:
            #batch.facts has the texts and batch.target has the labels.          
            self.optimizer.zero_grad()
            predictions = self.model(batch.facts).squeeze(1)
            loss = self.loss_fn(predictions, batch.target)                      
            accuracy = binary_accuracy(predictions, batch.target) 
                   
            loss.backward()
            self.optimizer.step()
                        
            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    def train_model(self, train_iterator, dev_iterator):
        dev_accs = [0.]
        for epoch in range(5):
            self.train_epoch(train_iterator)
            dev_acc = self.evaluate(dev_iterator)
            print(f"Epoch {epoch}: Dev Accuracy: {dev_acc[1]} Dev Loss:{dev_acc[0]}")
            if dev_acc[1] > max(dev_accs):
                best_model = copy.deepcopy(self)
            dev_accs.append(dev_acc[1])
        return best_model.model
                
    def evaluate(self, iterator):
        epoch_loss = 0
        epoch_acc = 0
        
        self.model.eval()
    
        with torch.no_grad():
            for batch in iterator:
                predictions = self.model(batch.facts).squeeze(1)
                loss = self.loss_fn(predictions, batch.target)                      
                accuracy = binary_accuracy(predictions, batch.target) 

                epoch_loss += loss.item()
                epoch_acc += accuracy.item()
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = WordEmbAvg(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)

In [ ]:
model = model.to()
tm = TrainingModule(model)

best_model = tm.train_model(train_iterator, valid_iterator)

Epoch 0: Dev Accuracy: 0.836552904030987 Dev Loss:0.3811733804852049
Epoch 1: Dev Accuracy: 0.8514060411497811 Dev Loss:0.34807823500900625
Epoch 2: Dev Accuracy: 0.856892523364486 Dev Loss:0.33633933606270316


In [ ]:
tm.model = best_model
test_loss, test_acc = tm.evaluate(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [12]:
a = tm.model.embedding.weight.data
result = list(torch.norm(a, p=2, dim=1).numpy())

word_ls = TEXT.vocab.itos
top_n = 50
top_neg = {}
top_pos = {}

max_pos = [result.index(x) for x in sorted(result, reverse=True)][:top_n]
max_neg = [result.index(x) for x in sorted(result)][:top_n]

for i in range(top_n):
    i_p = max_pos[i]
    i_n = max_neg[i]
    top_pos[word_ls[i_p]] = result[i_p]
    top_neg[word_ls[i_n]] = result[i_n]

In [13]:
top_neg

{'<pad>': 0.0,
 '340,000': 12.121225,
 'representing': 12.179583,
 'selected': 12.184908,
 'amenities': 12.235453,
 '05': 12.242419,
 'active': 12.246576,
 'march': 12.266368,
 '’': 12.281819,
 'elevator': 12.315381,
 'room': 12.321776,
 'under': 12.323661,
 'north': 12.339952,
 'life': 12.384555,
 '792': 12.392569,
 'significant': 12.397833,
 'inequitably': 12.43993,
 'response': 12.450005,
 'swift': 12.459846,
 'standing': 12.461824,
 'demolition': 12.468464,
 'found': 12.4745655,
 'owners': 12.485443,
 'non': 12.487463,
 '123': 12.520738,
 'tax': 12.530328,
 'bathrooms': 12.554637,
 'transaction': 12.5632105,
 'theboard': 12.580719,
 '2001': 12.586535,
 'subject.the': 12.590843,
 'replacement': 12.593771,
 '54': 12.605616,
 'closed': 12.6465645,
 'withinthe': 12.658865,
 '1,320': 12.664407,
 'argument': 12.666277,
 'single': 12.666827,
 '1967': 12.682098,
 '61': 12.687147,
 '320,000': 12.68996,
 '17': 12.691257,
 'kleszynski': 12.691625,
 '33.30': 12.691891,
 'private': 12.723279,
 

In [14]:
top_pos

{'submit': 18.00373,
 '1910.40(a': 17.746069,
 'limited': 17.55114,
 'dispute': 17.482466,
 'proposed': 17.475155,
 'complete': 17.473719,
 'fourequity': 17.470196,
 'valuation': 17.3974,
 'matter': 17.324213,
 'applied': 17.09348,
 'rebuttal': 17.076965,
 'answer': 17.061985,
 'timely': 17.06145,
 'address': 16.92053,
 'comparables.in': 16.863894,
 'applying': 16.76941,
 'insupport': 16.746136,
 'thebuilding': 16.73585,
 '1910.66(c': 16.692356,
 'depicted': 16.495459,
 'u.s': 16.486486,
 '105,000': 16.45333,
 'joseph': 16.421343,
 'stipulate': 16.41403,
 'comply': 16.408504,
 'determining': 16.38805,
 'madison': 16.369648,
 'differ': 16.361387,
 'said': 16.3195,
 'fairly': 16.316051,
 'overvalued': 16.293453,
 'triennial': 16.286564,
 'purported': 16.2607,
 'rollover': 16.243225,
 'pertaining': 16.237326,
 '1958': 16.212814,
 'multiple': 16.203047,
 'detached': 16.183487,
 'expense': 16.18097,
 'beginning': 16.170961,
 '441': 16.170826,
 'substantive': 16.166718,
 'subdivision': 16.16

# Alternate Evaluation Dimensions

In [15]:
joined = pd.merge(data_raw, decisions, left_on='docket', right_on='docket_name')
joined['year'] = joined.date.str.slice(0,4)

In [37]:
def eval_subset(joined, attr, val):
    bool_list = joined[attr] == val
    bool_list = bool_list.tolist()
    filtered_full = [i for (i, val) in zip(full.examples, bool_list) if val]

    mini_full = data.Dataset(
        filtered_full,
        fields=[('facts', TEXT),('target', LABEL)])

    filtered_iterator = data.Iterator(
        mini_full, 
        batch_size = BATCH_SIZE,
        device = 'cpu',
        sort_key=lambda x: len(x.facts))

    test_loss, test_acc = tm.evaluate(filtered_iterator)
    print("~~~~")
    print("Analyzing data where {} is {}".format(attr, val))
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
    print(f'Number of cases: {joined[bool_list].shape[0]}')
    print(f'Percent of cases with reductions: {joined[bool_list].target.mean()*100:.2f}%')

In [38]:
eval_subset(joined, 'county', 'Cook')

~~~~
Analyzing data where county is Cook
Test Loss: 0.294 | Test Acc: 88.00%
Number of cases: 36073
Percent of cases with reductions: 44.75%


In [41]:
eval_subset(joined, 'appellant', 'Mack Companies')

~~~~
Analyzing data where appellant is Mack Companies
Test Loss: 0.295 | Test Acc: 83.36%
Number of cases: 281
Percent of cases with reductions: 2.14%


In [42]:
eval_subset(joined, 'appellant', 'Inverclyde, LLC')

~~~~
Analyzing data where appellant is Inverclyde, LLC
Test Loss: 0.448 | Test Acc: 78.12%
Number of cases: 128
Percent of cases with reductions: 37.50%


In [47]:
eval_subset(joined, 'county', 'Lake')

~~~~
Analyzing data where county is Lake
Test Loss: 0.385 | Test Acc: 85.91%
Number of cases: 3487
Percent of cases with reductions: 14.11%


In [49]:
eval_subset(joined, 'reason_code', '2')

~~~~
Analyzing data where reason_code is 2
Test Loss: 0.318 | Test Acc: 86.97%
Number of cases: 42338
Percent of cases with reductions: 41.70%


In [50]:
eval_subset(joined, 'reason_code', '1')

~~~~
Analyzing data where reason_code is 1
Test Loss: 0.374 | Test Acc: 82.98%
Number of cases: 3195
Percent of cases with reductions: 40.85%


In [52]:
eval_subset(joined, 'prop_type', 'R')

~~~~
Analyzing data where prop_type is R
Test Loss: 0.326 | Test Acc: 86.59%
Number of cases: 41491
Percent of cases with reductions: 37.86%


In [53]:
eval_subset(joined, 'prop_type', 'C')

~~~~
Analyzing data where prop_type is C
Test Loss: 0.292 | Test Acc: 86.59%
Number of cases: 3230
Percent of cases with reductions: 79.13%


In [54]:
eval_subset(joined, 'prop_type', 'I')

~~~~
Analyzing data where prop_type is I
Test Loss: 0.241 | Test Acc: 91.26%
Number of cases: 847
Percent of cases with reductions: 83.12%


In [57]:
eval_subset(joined, 'valuation_class', 1)

~~~~
Analyzing data where valuation_class is 1
Test Loss: 0.321 | Test Acc: 86.72%
Number of cases: 45125
Percent of cases with reductions: 41.40%


In [58]:
eval_subset(joined, 'valuation_class', 2)

~~~~
Analyzing data where valuation_class is 2
Test Loss: 0.407 | Test Acc: 81.26%
Number of cases: 370
Percent of cases with reductions: 60.27%


In [68]:
eval_subset(joined, 'year', '2016')

~~~~
Analyzing data where year is 2016
Test Loss: 0.311 | Test Acc: 87.63%
Number of cases: 10473
Percent of cases with reductions: 50.55%


In [69]:
eval_subset(joined, 'year', '2017')

~~~~
Analyzing data where year is 2017
Test Loss: 0.324 | Test Acc: 86.47%
Number of cases: 8293
Percent of cases with reductions: 38.02%


In [70]:
eval_subset(joined, 'year', '2018')

~~~~
Analyzing data where year is 2018
Test Loss: 0.296 | Test Acc: 88.19%
Number of cases: 7399
Percent of cases with reductions: 30.42%


In [71]:
eval_subset(joined, 'year', '2019')

~~~~
Analyzing data where year is 2019
Test Loss: 0.328 | Test Acc: 86.71%
Number of cases: 7490
Percent of cases with reductions: 24.43%


In [72]:
eval_subset(joined, 'year', '2020')

~~~~
Analyzing data where year is 2020
Test Loss: 0.366 | Test Acc: 85.22%
Number of cases: 2721
Percent of cases with reductions: 22.79%
